<a href="https://colab.research.google.com/github/newmantic/Heston/blob/main/Heston.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

def heston_simulation(S0, v0, r, kappa, theta, sigma, rho, T, N, M):
    """
    Simulate asset prices using the Heston model.

    Parameters:
    - S0: Initial asset price
    - v0: Initial variance
    - r: Risk-free interest rate
    - kappa: Rate of mean reversion of variance
    - theta: Long-term mean of variance
    - sigma: Volatility of variance (volatility of volatility)
    - rho: Correlation between asset price and variance
    - T: Time to maturity
    - N: Number of time steps
    - M: Number of simulations

    Returns:
    - S: Simulated asset prices (M x N+1 array)
    """
    dt = T / N
    S = np.zeros((M, N + 1))
    v = np.zeros((M, N + 1))
    S[:, 0] = S0
    v[:, 0] = v0

    for t in range(1, N + 1):
        z1 = np.random.normal(size=M)
        z2 = np.random.normal(size=M)
        W1 = np.sqrt(dt) * z1
        W2 = rho * W1 + np.sqrt(1 - rho**2) * np.sqrt(dt) * z2

        v[:, t] = np.maximum(v[:, t-1] + kappa * (theta - v[:, t-1]) * dt + sigma * np.sqrt(v[:, t-1]) * W2, 0)
        S[:, t] = S[:, t-1] * np.exp((r - 0.5 * v[:, t-1]) * dt + np.sqrt(v[:, t-1]) * W1)

    return S

In [2]:
def heston_european_option_price(S0, K, r, T, S, option_type='call'):
    """
    Price a European option using the Heston model simulations.

    Parameters:
    - S0: Initial asset price
    - K: Strike price
    - r: Risk-free interest rate
    - T: Time to maturity
    - S: Simulated asset prices (M x N+1 array)
    - option_type: 'call' for call option, 'put' for put option

    Returns:
    - option_price: The price of the European option
    """
    M = S.shape[0]
    payoff = np.maximum(S[:, -1] - K, 0) if option_type == 'call' else np.maximum(K - S[:, -1], 0)
    option_price = np.exp(-r * T) * np.mean(payoff)

    return option_price

In [3]:
# Parameters
S0 = 100         # Initial asset price
v0 = 0.04        # Initial variance (square of initial volatility, e.g., 20% volatility = 0.04 variance)
r = 0.05         # Risk-free rate
kappa = 2.0      # Rate of mean reversion of variance
theta = 0.04     # Long-term mean of variance
sigma = 0.5      # Volatility of variance (volatility of volatility)
rho = -0.7       # Correlation between asset price and variance
T = 1            # Time to maturity (1 year)
N = 252          # Number of time steps (daily steps for 1 year)
M = 10000        # Number of simulations
K = 100          # Strike price

# Simulate asset prices
S = heston_simulation(S0, v0, r, kappa, theta, sigma, rho, T, N, M)

# Price the European call option
call_price = heston_european_option_price(S0, K, r, T, S, option_type='call')
print(f"Heston Model European Call Option Price: {call_price:.2f}")

Heston Model European Call Option Price: 10.26
